# Feature Engineering for Regime Identification

This notebook implements **Level**, **Trend**, and **Volatility** features from macroeconomic variables following the feature design document. These engineered features will form the input for PCA, clustering (K-Means/HMM), and similarity-based regime identification.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Load the macroeconomic data
DATA_PATH = '../data/processed/macro_data_1962on (cleaned).csv'
df = pd.read_csv(DATA_PATH, parse_dates=['date'])
df.set_index('date', inplace=True)
df = df.sort_index()

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Data shape: (766, 7)
Date range: 1962-01-31 00:00:00 to 2025-10-31 00:00:00

Columns: ['market', 'yield_curve', 'oil ($/bbl)', 'copper ($/metric ton)', 'monetary_policy', 'volatility', 'stock_bond_corr']


,market,yield_curve,oil ($/bbl),copper ($/metric ton),monetary_policy,volatility,stock_bond_corr
date,,,,,,,
1962-01-31,68.839996,-2.320,1.52,635.37,2.73,0.093924,NaN
1962-02-28,69.959999,-2.310,1.52,647.94,2.71,0.064570,NaN
1962-03-31,69.550003,-2.364,1.52,647.50,2.75,0.057723,NaN
1962-04-30,65.239998,-2.354,1.52,645.95,2.74,0.098421,NaN
1962-05-31,59.630001,-2.310,1.52,645.73,2.70,0.347108,NaN


## Feature Engineering Functions

### 1. Winsorization
Clip extreme values at the 1st and 99th percentiles to reduce the impact of outliers.

In [2]:
def winsorize(series: pd.Series, lower: float = 0.01, upper: float = 0.99) -> pd.Series:
    """
    Winsorize a series by clipping values at the specified percentiles.
    
    Parameters:
    -----------
    series : pd.Series
        Input time series
    lower : float
        Lower percentile for clipping (default: 1%)
    upper : float
        Upper percentile for clipping (default: 99%)
    
    Returns:
    --------
    pd.Series
        Winsorized series
    """
    lower_bound = series.quantile(lower)
    upper_bound = series.quantile(upper)
    return series.clip(lower=lower_bound, upper=upper_bound)

### 2. Level Feature
**Definition**: Normalized current value of a macro variable using expanding-window statistics.

$$\text{Level}_{t} = \frac{x_t - \mu_{1:t-1}}{\sigma_{1:t-1}}$$

Uses expanding mean and std up to time (t-1) to avoid lookahead bias.

In [3]:
def compute_level(series: pd.Series, min_periods: int = 12) -> pd.Series:
    """
    Compute the Level feature: expanding-window z-score.
    
    Level_t = (x_t - mean_{1:t-1}) / std_{1:t-1}
    
    Parameters:
    -----------
    series : pd.Series
        Input time series (should be winsorized)
    min_periods : int
        Minimum number of observations for expanding statistics
    
    Returns:
    --------
    pd.Series
        Level feature (z-scored)
    """
    # Winsorize the series
    series_w = winsorize(series)
    
    # Expanding mean and std, shifted by 1 to avoid lookahead bias
    expanding_mean = series_w.expanding(min_periods=min_periods).mean().shift(1)
    expanding_std = series_w.expanding(min_periods=min_periods).std().shift(1)
    
    # Compute z-score
    level = (series_w - expanding_mean) / expanding_std
    
    # Replace inf with NaN
    level = level.replace([np.inf, -np.inf], np.nan)
    
    return level

### 3. Trend Feature
**Definition**: Recent direction of a macro variable over the past N months.

Two options:
- **Slope**: OLS slope of variable over past 12 months
- **Momentum**: $(x_t / x_{t-12}) - 1$ for ratio-like variables, or $x_t - x_{t-12}$ for level variables

The result is then normalized using expanding z-score.

In [4]:
def rolling_ols_slope(series: pd.Series, window: int = 12) -> pd.Series:
    """
    Compute rolling OLS slope over a specified window.
    
    Parameters:
    -----------
    series : pd.Series
        Input time series
    window : int
        Rolling window size (default: 12 months)
    
    Returns:
    --------
    pd.Series
        Rolling OLS slope coefficients
    """
    slopes = []
    indices = series.index
    
    for i in range(len(series)):
        if i < window - 1:
            slopes.append(np.nan)
        else:
            y = series.iloc[i - window + 1:i + 1].values
            x = np.arange(window).reshape(-1, 1)
            
            # Handle missing values
            valid_mask = ~np.isnan(y)
            if valid_mask.sum() < 3:  # Need at least 3 points for regression
                slopes.append(np.nan)
            else:
                model = LinearRegression()
                model.fit(x[valid_mask], y[valid_mask])
                slopes.append(model.coef_[0])
    
    return pd.Series(slopes, index=indices)


def expanding_zscore(series: pd.Series, min_periods: int = 12) -> pd.Series:
    """
    Compute expanding z-score to normalize a series.
    
    Parameters:
    -----------
    series : pd.Series
        Input time series
    min_periods : int
        Minimum observations for expanding statistics
    
    Returns:
    --------
    pd.Series
        Expanding z-scored series
    """
    expanding_mean = series.expanding(min_periods=min_periods).mean().shift(1)
    expanding_std = series.expanding(min_periods=min_periods).std().shift(1)
    
    zscore = (series - expanding_mean) / expanding_std
    zscore = zscore.replace([np.inf, -np.inf], np.nan)
    
    return zscore


def compute_trend(series: pd.Series, method: str = "slope", window: int = 12) -> pd.Series:
    """
    Compute the Trend feature using OLS slope or momentum.
    
    Parameters:
    -----------
    series : pd.Series
        Input time series
    method : str
        'slope' for OLS regression slope, 'momentum' for percentage change
    window : int
        Lookback window for trend calculation (default: 12 months)
    
    Returns:
    --------
    pd.Series
        Trend feature (expanding z-scored)
    """
    # Winsorize the series
    series_w = winsorize(series)
    
    if method == "slope":
        # Rolling OLS slope
        trend_raw = rolling_ols_slope(series_w, window=window)
    elif method == "momentum":
        # Momentum: (x_t / x_{t-12}) - 1
        # Use pct_change for ratio, handle zeros
        trend_raw = series_w.pct_change(periods=window)
    else:
        raise ValueError(f"Method must be 'slope' or 'momentum', got {method}")
    
    # Normalize with expanding z-score
    trend = expanding_zscore(trend_raw)
    
    return trend

### 4. Volatility Feature
**Definition**: Historical variability of values over past N months.

$$\text{Vol}_t = \text{std}(x_{t-N+1}, ..., x_t)$$

The result is then normalized using expanding z-score.

In [5]:
def compute_volatility(series: pd.Series, window: int = 12) -> pd.Series:
    """
    Compute the Volatility feature: rolling std normalized by expanding z-score.
    
    Parameters:
    -----------
    series : pd.Series
        Input time series
    window : int
        Rolling window for volatility calculation (default: 12 months)
    
    Returns:
    --------
    pd.Series
        Volatility feature (expanding z-scored)
    """
    # Winsorize the series
    series_w = winsorize(series)
    
    # Rolling standard deviation
    vol_raw = series_w.rolling(window=window, min_periods=window).std()
    
    # Normalize with expanding z-score
    vol = expanding_zscore(vol_raw)
    
    return vol

### 5. Build Feature Matrix
Combine all Level, Trend, and Volatility features for each macro variable into a single feature matrix.

In [6]:
def build_feature_matrix(df: pd.DataFrame, 
                         trend_method: str = "slope",
                         window: int = 12,
                         exclude_cols: list = None) -> pd.DataFrame:
    """
    Build a complete feature matrix with Level, Trend, and Volatility features
    for each variable in the input dataframe.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe with macro variables (DatetimeIndex)
    trend_method : str
        Method for trend calculation ('slope' or 'momentum')
    window : int
        Lookback window for trend and volatility (default: 12 months)
    exclude_cols : list
        Columns to exclude from feature engineering
    
    Returns:
    --------
    pd.DataFrame
        Feature matrix with Level/Trend/Vol columns for each variable
    """
    if exclude_cols is None:
        exclude_cols = []
    
    feature_dfs = []
    
    for col in df.columns:
        if col in exclude_cols:
            continue
            
        series = df[col].dropna()
        
        if len(series) < window * 2:
            print(f"Skipping {col}: insufficient data points ({len(series)})")
            continue
        
        # Compute features
        level = compute_level(series)
        trend = compute_trend(series, method=trend_method, window=window)
        vol = compute_volatility(series, window=window)
        
        # Create feature dataframe
        features = pd.DataFrame({
            f'{col}_level': level,
            f'{col}_trend': trend,
            f'{col}_vol': vol
        })
        
        feature_dfs.append(features)
        print(f"Processed: {col}")
    
    # Concatenate all features
    feature_matrix = pd.concat(feature_dfs, axis=1)
    
    return feature_matrix

## Apply Feature Engineering to Macro Data

Generate the complete feature matrix from the macroeconomic variables.

In [7]:
# Build feature matrix using OLS slope for trend
feature_matrix = build_feature_matrix(df, trend_method="slope", window=12)

print(f"\nFeature matrix shape: {feature_matrix.shape}")
print(f"Feature columns: {feature_matrix.columns.tolist()}")

Processed: market
Processed: yield_curve
Processed: oil ($/bbl)
Processed: copper ($/metric ton)
Processed: monetary_policy
Processed: volatility
Processed: stock_bond_corr

Feature matrix shape: (766, 21)
Feature columns: ['market_level', 'market_trend', 'market_vol', 'yield_curve_level', 'yield_curve_trend', 'yield_curve_vol', 'oil ($/bbl)_level', 'oil ($/bbl)_trend', 'oil ($/bbl)_vol', 'copper ($/metric ton)_level', 'copper ($/metric ton)_trend', 'copper ($/metric ton)_vol', 'monetary_policy_level', 'monetary_policy_trend', 'monetary_policy_vol', 'volatility_level', 'volatility_trend', 'volatility_vol', 'stock_bond_corr_level', 'stock_bond_corr_trend', 'stock_bond_corr_vol']


In [8]:
# Preview the feature matrix
feature_matrix.head(30)

,market_level,market_trend,market_vol,yield_curve_level,yield_curve_trend,yield_curve_vol,oil ($/bbl)_level,oil ($/bbl)_trend,oil ($/bbl)_vol,copper ($/metric ton)_level,...,copper ($/metric ton)_vol,monetary_policy_level,monetary_policy_trend,monetary_policy_vol,volatility_level,volatility_trend,volatility_vol,stock_bond_corr_level,stock_bond_corr_trend,stock_bond_corr_vol
date,,,,,,,,,,,,,,,,,,,,,
1962-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Summary Statistics

In [9]:
# Summary statistics for features
print("Feature Summary Statistics (after burn-in period):\n")
feature_summary = feature_matrix.dropna().describe()
feature_summary

Feature Summary Statistics (after burn-in period):



,market_level,market_trend,market_vol,yield_curve_level,yield_curve_trend,yield_curve_vol,oil ($/bbl)_level,oil ($/bbl)_trend,oil ($/bbl)_vol,copper ($/metric ton)_level,...,copper ($/metric ton)_vol,monetary_policy_level,monetary_policy_trend,monetary_policy_vol,volatility_level,volatility_trend,volatility_vol,stock_bond_corr_level,stock_bond_corr_trend,stock_bond_corr_vol
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,...,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,2.309154,0.675532,1.752953,0.128677,0.046929,0.074908,1.396106,0.344590,1.142771,1.415337,...,0.579129,-0.115553,-0.046481,0.072889,1.583731,0.356206,1.022491,0.120364,-0.033204,-0.111693
std,1.173872,2.174073,1.669238,1.366326,1.154815,1.201851,1.879416,2.275940,2.013891,1.379893,...,1.523854,1.374436,1.155418,1.204982,12.925547,5.296860,6.617258,1.261524,1.041540,1.099188
min,-1.935268,-6.917993,-1.896783,-4.378967,-3.876288,-1.174504,-2.810632,-9.613419,-1.837463,-0.655312,...,-1.280634,-2.051086,-5.003234,-1.197677,-0.773121,-6.549487,-1.761487,-2.107876,-3.641215,-2.539471
25%,1.537850,-0.488424,0.528579,-0.524212,-0.548727,-0.849879,0.382666,-0.671756,0.049724,0.191002,...,-0.450530,-1.277077,-0.582869,-0.864393,0.218125,-0.642840,-0.176969,-0.956637,-0.654045,-0.840047
50%,2.481644,0.783666,1.489958,0.363552,-0.000069,-0.318210,1.222843,0.018832,0.555748,1.312911,...,0.219230,-0.346138,0.000672,-0.298580,0.760614,0.001204,0.416299,0.017277,0.033858,-0.382238
75%,3.122974,1.912155,2.767907,1.315436,0.577434,0.673694,2.527460,1.035563,1.617965,2.171380,...,1.121049,0.580452,0.547187,0.665728,1.703963,0.796741,1.353515,1.030878,0.639488,0.286495
max,4.538464,6.969533,6.662849,2.068146,5.139960,4.313497,24.286860,17.359319,24.186234,8.528348,...,9.056869,4.390596,3.948147,4.378271,348.021112,131.220359,172.310792,3.451362,2.914009,4.089065


In [10]:
# Check missing values
print("Missing values per feature:\n")
missing_counts = feature_matrix.isna().sum()
print(missing_counts)

# Valid data coverage
valid_rows = feature_matrix.dropna().shape[0]
total_rows = feature_matrix.shape[0]
print(f"\nValid rows (no missing): {valid_rows} / {total_rows} ({100*valid_rows/total_rows:.1f}%)")
print(f"Date range with complete features: {feature_matrix.dropna().index.min()} to {feature_matrix.dropna().index.max()}")

Missing values per feature:

market_level                   12
market_trend                   23
market_vol                     23
yield_curve_level              12
yield_curve_trend              23
yield_curve_vol                23
oil ($/bbl)_level              13
oil ($/bbl)_trend              23
oil ($/bbl)_vol                23
copper ($/metric ton)_level    12
copper ($/metric ton)_trend    23
copper ($/metric ton)_vol      23
monetary_policy_level          12
monetary_policy_trend          23
monetary_policy_vol            23
volatility_level               12
volatility_trend               23
volatility_vol                 23
stock_bond_corr_level          24
stock_bond_corr_trend          35
stock_bond_corr_vol            35
dtype: int64

Valid rows (no missing): 731 / 766 (95.4%)
Date range with complete features: 1964-12-31 00:00:00 to 2025-10-31 00:00:00


### Why are there missing values?

The missing values come from the **burn-in period** required for window-based calculations:

| Feature | Source of NaNs | Count |
|---------|---------------|-------|
| **Level** | `expanding(min_periods=12).shift(1)` → need 12 obs + 1 shift | **12** |
| **Trend** | `rolling(window=12)` → 11 NaN, then `expanding(min_periods=12).shift(1)` → 12 more | **23** |
| **Volatility** | `rolling(window=12)` → 11 NaN, then `expanding(min_periods=12).shift(1)` → 12 more | **23** |

**Note**: `stock_bond_corr` has additional missing values (24/35) because the raw data itself is missing for the first 12 months (1962).

In [11]:
# Visualize the burn-in period
print("First valid values for each feature type (market example):\n")
market_features = feature_matrix[['market_level', 'market_trend', 'market_vol']]
first_valid = market_features.apply(lambda x: x.first_valid_index())
print(first_valid)

print("\n\nBreakdown of missing value sources:")
print("=" * 50)
print("Level:  12 NaN = min_periods(12) + shift(1) - 1")
print("Trend:  23 NaN = rolling(12) gives 11 NaN + expanding z-score adds 12")  
print("Vol:    23 NaN = rolling(12) gives 11 NaN + expanding z-score adds 12")
print("\nstock_bond_corr has +12 extra NaN because raw data starts in 1963")

First valid values for each feature type (market example):

market_level   1963-01-31
market_trend   1963-12-31
market_vol     1963-12-31
dtype: datetime64[ns]


Breakdown of missing value sources:
Level:  12 NaN = min_periods(12) + shift(1) - 1
Trend:  23 NaN = rolling(12) gives 11 NaN + expanding z-score adds 12
Vol:    23 NaN = rolling(12) gives 11 NaN + expanding z-score adds 12

stock_bond_corr has +12 extra NaN because raw data starts in 1963


## Save Feature Matrix

Export the engineered features for use in regime modeling.

In [12]:
# Save the complete feature matrix
output_path = '../data/processed/feature_matrix.csv'
feature_matrix.to_csv(output_path)
print(f"Feature matrix saved to: {output_path}")

# Also save a version with only complete rows (no NaN)
feature_matrix_clean = feature_matrix.dropna()
output_path_clean = '../data/processed/feature_matrix_clean.csv'
feature_matrix_clean.to_csv(output_path_clean)
print(f"Clean feature matrix saved to: {output_path_clean}")
print(f"Clean matrix shape: {feature_matrix_clean.shape}")

Feature matrix saved to: ../data/processed/feature_matrix.csv
Clean feature matrix saved to: ../data/processed/feature_matrix_clean.csv
Clean matrix shape: (731, 21)
